In [15]:

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [16]:

headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [17]:
product_url=input()
# https://www.amazon.com/Invicta-8926OB-Stainless-Automatic-Bracelet/dp/B000JQFX1G/?_encoding=UTF8&pd_rd_w=oivdj&content-id=amzn1.sym.6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_p=6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_r=B6N0XX1NXHP4ZV6JZ6RQ&pd_rd_wg=4JYln&pd_rd_r=7c96c10f-2150-47d9-bdc2-27cabcad19ba&ref_=pd_hp_d_btf_nta-top-sellers&th=1



 https://www.amazon.com/Invicta-8926OB-Stainless-Automatic-Bracelet/dp/B000JQFX1G/?_encoding=UTF8&pd_rd_w=oivdj&content-id=amzn1.sym.6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_p=6cf7c564-ca00-466b-86d0-47197a966847&pf_rd_r=B6N0XX1NXHP4ZV6JZ6RQ&pd_rd_wg=4JYln&pd_rd_r=7c96c10f-2150-47d9-bdc2-27cabcad19ba&ref_=pd_hp_d_btf_nta-top-sellers&th=1


In [18]:
import re


def extract_product_info(url):
    comf=False
   
    pattern = r"amazon\.in/([^/]+)/dp/([^/?]+)" 

        
    
   
    match = re.search(pattern, url)
    if not match:
        pattern=r"amazon\.com/([^/]+)/dp/([^/?]+)" 
        match = re.search(pattern, url)
        comf=True
        
    
    if match:
        product_name = match.group(1)
        product_id = match.group(2)
        return product_name, product_id,comf
    else:
        return None, None,True





In [19]:

product_name, product_id,comf = extract_product_info(product_url)

if product_name and product_id:
    print(f"Product Name: {product_name}")
    print(f"Product ID: {product_id}")
else:
    print("Product information not found.")

Product Name: Invicta-8926OB-Stainless-Automatic-Bracelet
Product ID: B000JQFX1G


In [20]:

reviews_url = 'https://www.amazon.com/'+product_name+'/product-reviews/'+product_id

    

In [21]:
reviews_url

'https://www.amazon.com/Invicta-8926OB-Stainless-Automatic-Bracelet/product-reviews/B000JQFX1G'

In [22]:

len_page = 4

In [23]:

def reviewsHtml(url, len_page):
    
    
    soups = []
    
    
    for page_no in range(1, len_page + 1):
        
        
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        
        response = requests.get(url, headers=headers)
        
      
        soup = BeautifulSoup(response.text, 'lxml')
        
        
        soups.append(soup)
        
    return soups

In [24]:

def getReviews(html_data):

    data_dicts = []
    
    
    boxes = html_data.select('div[data-hook="review"]')
    
     
    for box in boxes:
        
      
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
           
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        
        data_dict = {
           
            'Stars' : stars,
            'Description' : description
        }

        
        data_dicts.append(data_dict)
    
    return data_dicts

In [25]:

html_datas = reviewsHtml(reviews_url, len_page)

In [26]:

reviews = []

In [27]:

for html_data in html_datas:
    
   
    review = getReviews(html_data)
    
  
    reviews += review

In [32]:

df_reviews = pd.DataFrame(reviews)
df_reviews

,Name,Stars,Title,Date,Description
0,Mr. Manalyst,5.0,5.0 out of 5 stars\nHave no fear -- it IS the ...,18/01/2016,"Frankly, I was somewhat skeptical. A decent, l..."
1,Laxluver1234,5.0,5.0 out of 5 stars\nI've done my research - We...,19/07/2024,This has been my first automatic type watch pu...
2,ezwip,5.0,5.0 out of 5 stars\nImpressive,20/04/2024,First I'll just say the one I received did not...
3,Joehl Crawford-Robinson,4.0,4.0 out of 5 stars\nAffordable Automatic Diver,30/04/2024,Invicta gets a lot of hate from the watch comm...
4,OA,5.0,"5.0 out of 5 stars\nVery solid, beautiful mech...",03/01/2024,"For the price, this is amazing!To get it out o..."
5,J. R. Bob Dobbs (formerly ZOMG Pink Ponies),5.0,"5.0 out of 5 stars\nBought as a joke, but actu...",08/12/2022,When I saw this watch on the Black Friday door...
6,Richmond,5.0,5.0 out of 5 stars\nGot a good one,15/12/2014,After having heard so many negative things abo...
7,Matt,5.0,5.0 out of 5 stars\nNo Better Watch for the Mo...,24/02/2014,"I received this watch about a week ago, and I ..."
8,Gustavo Chaparro,5.0,"5.0 out of 5 stars\nINVICTA 8926 OB, un reloj ...",17/04/2023,Escribo esta reseña en ESPAÑOL porque casi tod...
9,Bill Branscum,5.0,5.0 out of 5 stars\n8926 vs the Ubiquitous 993...,13/05/2021,My first dive watch was Seiko's first dive wat...


In [20]:

df_reviews.to_csv('reviews.csv', index=False)

In [31]:
len(df_reviews)

40